In [ ]:
#@markdown # Use Google Drive (optional)
#@markdown Models will be saved and loaded from your Google Drive.
from google.colab import drive
mount_google_drive = True
drive.mount('/content/gdrive')

In [ ]:
#@markdown # Download repo
update_repo = False #@param {type:"boolean"}

if not "mount_google_drive" in locals():
  mount_google_drive = False

if mount_google_drive:
  %cd /content/gdrive/MyDrive/
else:
  %cd /content/

if update_repo:
  %cd nataili/
  !git pull
else:
  !git clone https://github.com/sd-webui/nataili.git

In [ ]:
import json, requests, time, os
from google.colab import drive
from IPython.display import clear_output

if not "mount_google_drive" in locals():
  mount_google_drive = False

#@markdown # Model Download/Load
token = "" #@param {type:"string"}

#@markdown Or
Path_to_trained_model = "" #@param {type:"string"}
#@markdown Insert the full path of your trained model (eg: /content/gdrive/MyDrive/zarathustra.ckpt) and it will automatically be placed in the right place, otherwise, leave it EMPTY (make sure there are no spaces in the path)

#@markdown ---

stable_diffusion_v1_5 = True #@param {type:"boolean"}
#@markdown * Generalist AI image generating model. The baseline for all finetuned models.
stable_diffusion_v1_4 = False #@param {type:"boolean"}
#@markdown * Old version. Generalist AI image generating model. The baseline for all finetuned models.
waifu_diffusion = False #@param {type:"boolean"}
#@markdown * Anime styled generations.
furry_epoch = False #@param {type:"boolean"}
#@markdown * Furry styled generations.
yiffy = False #@param {type:"boolean"}
#@markdown * Furry styled generations.
Zack3D = False #@param {type:"boolean"}
#@markdown * Kink/NSFW oriented furry styled generations.
trinart = False #@param {type:"boolean"}
#@markdown * SFW Manga styled generations.

if mount_google_drive:
  %cd /content/gdrive/MyDrive/nataili/
else:
  %cd /content/nataili/

!mkdir -p models/custom/
!mkdir -p models/ldm/stable-diffusion-v1/

models = json.load(open('./db.json'))
dependencies = json.load(open('./db_dep.json'))
remote_models = "https://raw.githubusercontent.com/db0/nataili-model-reference/main/db.json"
remote_dependencies = "https://raw.githubusercontent.com/db0/nataili-model-reference/main/db_dep.json"
try:
  r = requests.get(remote_models)
  models = r.json()
  r = requests.get(remote_dependencies)
  dependencies = dependencies
except:
  models = models
  dependencies = dependencies

def download_model(name):
  if not name in models:
    print("Model not found!")

  model = models[name]
  download = model["config"]["download"][0]

  if 'file_url' in download:
    download_url = download['file_url']
    link = download_url.split("/")
    file_name = link[len(link) - 1]
    if 'hf_auth' in download:
      download_url = download_url.format(username="USER", password=token)

  if token == "" and "hf_auth" in download:
    print('Huggingface token not supplied!')
    exit()

  model_destination = model["config"]["files"][0]["path"]

  !wget -O $model_destination $download_url
  print("Downloaded model:", name)

if stable_diffusion_v1_5:
  download_model("stable_diffusion")
if stable_diffusion_v1_4:
  download_model("stable_diffusion_1.4")
if waifu_diffusion:
  download_model("waifu_diffusion")
if furry_epoch:
  download_model("waifu_diffusion")
if yiffy:
  download_model("Yiffy")
if Zack3D:
  download_model("Zack3D")
if trinart:
  download_model("trinart")

if Path_to_trained_model != '':
    if os.path.exists(str(Path_to_trained_model)):
        clear_output()
        !cp $Path_to_trained_model $required_path
        if os.path.exists(required_path):
            print('Model placed in the right directory')
        else:
            print('Something went wrong')
    else:
        print('Wrong path, use the colab file explorer to copy the path')

In [ ]:
#@markdown # Download dependencies (run again if there's an error)
if not "mount_google_drive" in locals():
  mount_google_drive = False

%cd /content/
!wget -qO- https://micromamba.snakepit.net/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
if mount_google_drive:
  !bin/micromamba create --no-shortcuts -r conda -n linux -f gdrive/MyDrive/nataili/environment.yaml -y
else:
  !bin/micromamba create --no-shortcuts -r conda -n linux -f nataili/environment.yaml -y

# Setting worker info
1. Rename `bridgeData_template.py` to `bridgeData.py`. 
2. Edit `bridgeData.py` and enter details such as the `horde_name` and `horde_api_key` you've received, so that you can receive Kudos.
3. If you downloaded extra models other than 1.5, list it in `models_to_load`.
**Note**: If you do not do this step, you will contribute anonymously.

In [ ]:
#@markdown # Start Stable Horde worker
if not "mount_google_drive" in locals():
  mount_google_drive = False

if mount_google_drive:
  %cd /content/gdrive/MyDrive/nataili/
else:
  %cd /content/nataili/
!/content/bin/micromamba run -r /content/conda -n linux python bridge.py $*